In [2]:
pip install openai --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Tool which can extract text from pdf
# pip install openai PyMuPDF pdfplumber pandas

In [4]:
import openai
print(openai.__version__)


1.35.3


In [28]:
import os
import openai
from openai import OpenAI
api_key = "OpenAI APIkey"
client = OpenAI(api_key = api_key)

def get_completion(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response


result = get_completion("how is weather in atlanta today?")
print(result)



ChatCompletion(id='chatcmpl-9diDfiMwkoKACSeWh49CQXleexwQZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, I do not have the ability to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Atlanta.", role='assistant', function_call=None, tool_calls=None))], created=1719251799, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=39, prompt_tokens=15, total_tokens=54))


In [21]:
import os
import time
from openai import OpenAI, OpenAIError
api_key = "OpenAI APIkey"

# Initialize the OpenAI client
client = OpenAI(api_key=api_key)

# Define the usage limits
MAX_TPM = 39999  # Tokens per minute
MAX_RPM = 3      # Requests per minute
MAX_RPD = 199    # Requests per day
MAX_TPD = 199999 # Tokens per day

# Track usage
requests_per_minute = 0
requests_per_day = 0
tokens_per_minute = 0
tokens_per_day = 0
start_time = time.time()
day_start_time = time.time()

def create_chat_completion(client, messages, model="gpt-3.5-turbo", retries=5):
    global requests_per_minute, requests_per_day, tokens_per_minute, tokens_per_day, start_time, day_start_time
    # had maximum rate  problem
    for i in range(retries):
        try:
            current_time = time.time()
            
            # Reset counters if a minute or a day has passed
            if current_time - start_time >= 60:
                requests_per_minute = 0
                tokens_per_minute = 0
                start_time = current_time
            if current_time - day_start_time >= 86400:
                requests_per_day = 0
                tokens_per_day = 0
                day_start_time = current_time

            if requests_per_minute >= MAX_RPM or requests_per_day >= MAX_RPD:
                raise Exception("Rate limit exceeded")
            if tokens_per_minute + len(messages) * 4 > MAX_TPM or tokens_per_day + len(messages) * 4 > MAX_TPD:  # Rough estimate: 4 tokens per word
                raise Exception("Token limit exceeded")
            
            # Make the API request
            response = client.chat.completions.create(
                messages=messages,
                model=model
            )
            
            # counters update
            requests_per_minute += 1
            requests_per_day += 1
            tokens_used = sum(len(message['content'].split()) for message in messages) * 4  # Rough estimate: 4 tokens per word
            tokens_per_minute += tokens_used
            tokens_per_day += tokens_used
            
            return response
        except OpenAIError as e:
            # Catch all OpenAI errors including rate limits
            wait_time = 2 ** i  # Exponential backoff
            print(f"OpenAI error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except Exception as e:
            # Handle custom rate limit and token limit errors
            print(e)
            time.sleep(60)  # Wait for a minute before retrying
    raise Exception("Exceeded maximum retries")

# Define the conversation messages
conversation = [
    {"role": "user", "content": "Area 51"}
]

# Make the request with retry mechanism
response = create_chat_completion(client, conversation)

# Print the response
print(response)


ChatCompletion(id='chatcmpl-9di2BpIhiRoeAYbEDc5mHwhaCKa0h', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Area 51, located in Nevada, USA, is a highly classified US Air Force facility that has been the subject of many conspiracy theories and speculation. It is known for its secrecy and reportedly houses experimental aircraft and weapons testing. The facility has been linked to various conspiracy theories, including the presence of extraterrestrial technology and captured alien beings. Despite its secretive nature, the US government has acknowledged the existence of Area 51 and its role in developing military technology.', role='assistant', function_call=None, tool_calls=None))], created=1719251087, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=92, prompt_tokens=10, total_tokens=102))


In [27]:
import os
import time
from openai import OpenAI, OpenAIError


# Initialize the OpenAI client
client = OpenAI(api_key=api_key)


def create_chat_completion(client, messages, model="gpt-3.5-turbo"):
        response = client.chat.completions.create(
                        messages=messages,
                        model=model
                    )
        return response


conversation = [
    {"role": "user", "content": "Area 51"}
]

# Make the request with retry mechanism
response = create_chat_completion(client, conversation)

# Print the response
print(response)


ChatCompletion(id='chatcmpl-9diCW7vLCrIZYOaNC7RlMX50mYZix', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Area 51 is a highly classified United States Air Force facility located in the Nevada desert. It is known for its secrecy and speculation surrounding its purpose, with many believing it is used for the testing and development of experimental aircraft and technology. The facility has also been the subject of various conspiracy theories and urban legends, with some claiming it is used for extraterrestrial research and the storage of alien spacecraft. While the true purpose of Area 51 remains undisclosed, it continues to capture the imagination of the public and inspire curiosity about what may be hidden within its walls.', role='assistant', function_call=None, tool_calls=None))], created=1719251728, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_t